<a href="https://colab.research.google.com/github/YazCodes/y2k_data_model/blob/main/y2k_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files
uploaded = files.upload()


Saving chat_data.txt to chat_data (1).txt


In [10]:
!ls


'chat_data (1).txt'   chat_data.txt   sample_data


In [11]:
!pip install transformers datasets peft accelerate bitsandbytes


In [12]:
from datasets import load_dataset

dataset = load_dataset('text', data_files={'train': 'chat_data.txt'})
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 185
    })
})

In [18]:
from transformers import AutoTokenizer # turns words into tokens

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token # ensures all inputs have the same lenght

def tokenize_function(examples): # a function converting raw text (the training data) into tokens (for the model to understand)
  tokenized_inputs = tokenizer(
      examples["text"],
      truncation =True,
      padding = "max_length",
      max_length = 128
  )
  tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy() # Add labels for training
  return tokenized_inputs

# applying the tokenize_function to every row of my dataset

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(model_name)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # GPT2 attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./y2k_model",
    # evaluation_strategy="no", # Removed unsupported argument
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

trainer.train()

In [22]:
trainer.save_model("y2k_model")
tokenizer.save_pretrained("y2k_model")


('y2k_model/tokenizer_config.json',
 'y2k_model/special_tokens_map.json',
 'y2k_model/vocab.json',
 'y2k_model/merges.txt',
 'y2k_model/added_tokens.json',
 'y2k_model/tokenizer.json')

In [ ]:
# testing the model

In [33]:
from transformers import pipeline

generator = pipeline("text-generation", model="y2k_model", tokenizer=tokenizer)

prompt = "User: What accessories for a Y2K outfit?\nBot:"
print(generator(prompt, max_length=80, do_sample=True, temperature=0.8)[0]["generated_text"])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


User: What accessories for a Y2K outfit?
Bot: Y2K outfits come with a black jacket, a scarf, and a short skirt.


In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
import transformers
print(transformers.__file__)


In [19]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 6.4 MB/s eta 0:00:00


In [21]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    args=training_args,
    # max_seq_length=128, # Removed unsupported argument
)

trainer.train()

Truncating train dataset:   0%|          | 0/185 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=141, training_loss=0.9519395760610594, metrics={'train_runtime': 585.1831, 'train_samples_per_second': 0.948, 'train_steps_per_second': 0.241, 'total_flos': 18300304097280.0, 'train_loss': 0.9519395760610594})